In [80]:
import pandas as pd
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
census = pd.read_csv("census.csv")

Vamos discriminar quais  (excetuando o income que é a classe a ser prevista) foram mais importantes depois da aplicação dos algoritmos de Seleção.

# Pré-processamento dos dados  
- Separação da base X, y
- Label Encoder
- Padronização dos dados com MinMaxScaler

In [3]:
colunas = census.columns[:-1] # todos os labels exceto income

X: np.ndarray = census.iloc[:, 0:14].values 
y: np.ndarray = census.iloc[: ,14].values 

### Label Enconder
encoder_workclass = LabelEncoder()
encoder_education = LabelEncoder()
encoder_marital= LabelEncoder()
encoder_occupation = LabelEncoder()
encoder_relationship = LabelEncoder()
encoder_race = LabelEncoder()
encoder_sex = LabelEncoder()
encoder_country = LabelEncoder()


X[:,1] = encoder_workclass.fit_transform(X[:,1])
X[:,3] = encoder_education.fit_transform(X[:,3])
X[:,5] = encoder_marital.fit_transform(X[:,5])
X[:,6] = encoder_occupation.fit_transform(X[:,6])
X[:,7] = encoder_relationship.fit_transform(X[:,7])
X[:,8] = encoder_race.fit_transform(X[:,8])
X[:,9] = encoder_sex.fit_transform(X[:,9])
X[:,13] = encoder_country.fit_transform(X[:,13])

X

array([[39, 7, 77516, ..., 0, 40, 39],
       [50, 6, 83311, ..., 0, 13, 39],
       [38, 4, 215646, ..., 0, 40, 39],
       ...,
       [58, 4, 151910, ..., 0, 40, 39],
       [22, 4, 201490, ..., 0, 20, 39],
       [52, 5, 287927, ..., 0, 40, 39]], dtype=object)

In [ ]:
### Padronizacao
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[0.30136986, 0.875     , 0.0443019 , ..., 0.        , 0.39795918,
        0.95121951],
       [0.45205479, 0.75      , 0.0482376 , ..., 0.        , 0.12244898,
        0.95121951],
       [0.28767123, 0.5       , 0.13811345, ..., 0.        , 0.39795918,
        0.95121951],
       ...,
       [0.56164384, 0.5       , 0.09482688, ..., 0.        , 0.39795918,
        0.95121951],
       [0.06849315, 0.5       , 0.12849934, ..., 0.        , 0.19387755,
        0.95121951],
       [0.47945205, 0.625     , 0.18720338, ..., 0.        , 0.39795918,
        0.95121951]])

In [84]:
X_scaled.shape, y

((32561, 14),
 array([' <=50K', ' <=50K', ' <=50K', ..., ' <=50K', ' <=50K', ' >50K'],
       dtype=object))

# Low Variance  

In [35]:
numero_atributos: int = X.shape[1] # 14 atributos
columns_variances = [X_scaled[:,i].var() for i in range(numero_atributos)] 
columns_variances = np.array(columns_variances)

In [38]:

columns_variances.min(), columns_variances.max(), columns_variances.mean()

(0.005138537590667898, 0.22136950173699113, 0.05424126929314369)

In [ ]:
THRESHOLD = 0.05
columns_variances > THRESHOLD  # Serão selecionados apenas as colunas True

array([False, False, False,  True, False,  True,  True,  True, False,
        True, False, False, False, False])

In [57]:
indices_selecionados = np.where(columns_variances > THRESHOLD)
indices_selecionados

(array([3, 5, 6, 7, 9], dtype=int64),)

In [59]:
colunas[indices_selecionados]


Index(['education', 'marital-status', 'occupation', 'relationship', 'sex'], dtype='object')

O Threshold é a variância mínima a ser aceita. No caso, buscamos aqueles atributos que tenham variância alta. No nosso exemplo, as duas primeiras colunas estão em papel semelhante na nossa base de dados pois suas variâncias são semelhantes. Uma variância muito pequena significa que os valores estão muito próximos da média, então todos os valores são parecidos entre si. Buscamos então atributos que tenham uma variância **MAIOR**.  

In [53]:
### O Threshold é a variância mínima

selecao = VarianceThreshold(threshold=THRESHOLD) # Feature selector that removes all low-variance features.
X_variancia = selecao.fit_transform(X_scaled)

### Criação da nova base de dados 

In [71]:
colunas[indices_selecionados]

Index(['education', 'marital-status', 'occupation', 'relationship', 'sex'], dtype='object')

In [72]:
columns_to_drop: list[str] = ['age', 'workclass', 'final-weight', 'education-num', 'race', 'capital-gain', 'capital-loos', 'hour-per-week', 'native-country']

census_var = census.drop(columns=columns_to_drop, axis=1)
census_var

,education,marital-status,occupation,relationship,sex,income
0,Bachelors,Never-married,Adm-clerical,Not-in-family,Male,<=50K
1,Bachelors,Married-civ-spouse,Exec-managerial,Husband,Male,<=50K
2,HS-grad,Divorced,Handlers-cleaners,Not-in-family,Male,<=50K
3,11th,Married-civ-spouse,Handlers-cleaners,Husband,Male,<=50K
4,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Female,<=50K
...,...,...,...,...,...,...
32556,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,Female,<=50K
32557,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,Male,>50K
32558,HS-grad,Widowed,Adm-clerical,Unmarried,Female,<=50K
32559,HS-grad,Never-married,Adm-clerical,Own-child,Male,<=50K


### Pré-processamento na base nova

In [77]:

X: np.ndarray = census_var.iloc[:, 0:5].values 
y: np.ndarray = census_var.iloc[: ,5].values 

### Label Enconder
encoder_education = LabelEncoder()
encoder_marital= LabelEncoder()
encoder_occupation = LabelEncoder()
encoder_relationship = LabelEncoder()
encoder_sex = LabelEncoder()


X[:,0] = encoder_education.fit_transform(X[:,0])
X[:,1] = encoder_marital.fit_transform(X[:,1])
X[:,2] = encoder_occupation.fit_transform(X[:,2])
X[:,3] = encoder_relationship.fit_transform(X[:,3])
X[:,4] = encoder_sex.fit_transform(X[:,4])

print('Base com label encoder:\n', X)

### One Hot Encoder
transformers: list[tuple] = [("OneHot", OneHotEncoder(), [0,1,2,3,4])]

OHE = ColumnTransformer(transformers=transformers, remainder="passthrough")
X = OHE.fit_transform(X).toarray()

print('Base com One Hot Encoder:\n', X)


X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.15, random_state=0)

Base com label encoder:
 [[9 4 1 1 1]
 [9 2 4 0 1]
 [11 0 6 1 1]
 ...
 [11 6 1 4 0]
 [11 4 1 3 1]
 [11 2 4 5 0]]
Base com One Hot Encoder:
 [[0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 1. 1. 0.]]


In [79]:
X_train.shape, X_test.shape

((27676, 46), (4885, 46))

Reduzimos o tamanho de 108 colunas para 46 colunas após a redução da dimensionalidade da base com essa base nova.

**Verificando o impacto em uma classificação por random forest**

In [81]:
### Tinha obitdo 84.7% com dados desbalanceados

rf_clf = RandomForestClassifier(
    criterion='entropy',
    min_samples_leaf=1,
    min_samples_split=5,
    n_estimators=100
)

rf_clf.fit(X_train, y_train)

y_pred = rf_clf.predict(X=X_test)
accuracy_score(y_true=y_test, y_pred=y_pred)

0.8176049129989764

O _accuracy_ não subiu tanto pois temos poucos atributos na base original, apenas saímos de 14 para 6. Funcionaria melhor em bases maiores com 1000 atributos indo para 700, por exemplo.